In [1]:
import sys
import higra as hg
sys.path.append('../../evapo_scripts/Auxiliary/')
from helpsters import *


In [3]:
# Function from https://github.com/waldnerf/decode/blob/main/postprocessing/instance_segmentation.py

def InstSegm(extent, boundary, t_ext=0.4, t_bound=0.2):
    """
    INPUTS:
    extent : extent prediction
    boundary : boundary prediction
    t_ext : threshold for extent
    t_bound : threshold for boundary
    OUTPUT:
    instances
    """

    # Threshold extent mask
    ext_binary = np.uint8(extent >= t_ext)

    # Artificially create strong boundaries for
    # pixels with non-field labels
    input_hws = np.copy(boundary)
    input_hws[ext_binary == 0] = 1

    # Create the directed graph
    size = input_hws.shape[:2]
    graph = hg.get_8_adjacency_graph(size)
    edge_weights = hg.weight_graph(
        graph,
        input_hws,
        hg.WeightFunction.mean)

    tree, altitudes = hg.watershed_hierarchy_by_dynamics(
        graph,
        edge_weights)

    # Get individual fields
    # by cutting the graph using altitude
    instances = hg.labelisation_horizontal_cut_from_threshold(
        tree,
        altitudes,
        threshold=t_bound).astype(float)

    instances[ext_binary == 0] = np.nan

    return instances

def exportNPtoVRTextent(array, vrt, outPath):
    '''A numpy array and vrt/tif file or a path to such have to provided. The numpy array should be derived and quite likely manipulated 
    from the provided vrt/tif to match its spatial extent.
    A path in the form of  path/to/file.tif for storing the created tif has to be provided'''

    if type(vrt) == str:
        vrt = gdal.Open(vrt)
    gtiff_driver = gdal.GetDriverByName('GTiff')
    if len(array.shape) == 2:
        nobands = 1
    else:
        nobands = array.shape[2]
    
    out_ds = gtiff_driver.Create(outPath, array.shape[1], array.shape[0], nobands, gdal.GDT_Int32)
    out_ds.SetGeoTransform(vrt.GetGeoTransform())
    out_ds.SetProjection(vrt.GetProjection())
    
    if nobands > 1:
        for i in range(nobands):
            out_ds.GetRasterBand(i + 1).WriteArray(array[:, :, i])
    else:
        out_ds.GetRasterBand(1).WriteArray(array[:, :])
    del out_ds

    print('Tif created')

def vectorizeRaster(rasterPath, outPath):
    '''vectorizes a raster. The first band of a raster at rasterpath will be vectorized and stored at outPath'''
    ds = gdal.Open(rasterPath)
    b1 = ds.GetRasterBand(1)

    driver = ogr.GetDriverByName("ESRI Shapefile")
    vector_ds = driver.CreateDataSource(outPath)
    layer = vector_ds.CreateLayer("layer", srs=getSpatRefRas(ds), geom_type=ogr.wkbPolygon)
    
    # field for 'Field ID' from segmentation
    field = ogr.FieldDefn("Value", ogr.OFTInteger)
    layer.CreateField(field)

    gdal.Polygonize(b1, None, layer, 0, [], callback=None)

    ds = None
    vector_ds = None

    print("Raster successfully vectorized and stored at:", outPath)

In [ ]:
# load masked and unmasked .vrt files and run InstSegm algorythm

size = str(128*4)
overlap = 80
paths = [f'/data/Aldhani/output/predictions/FORCE/vrt/{size}_{overlap}_chipsvrt.vrt',
         f'/data/Aldhani/output/predictions/FORCE/vrt/{size}_{overlap}_masked_chipsvrt.vrt']

for path in paths:
    
    ds = gdal.Open(path)
    bandNumber = ds.RasterCount
    bands = []
    for i in range(bandNumber):
        bands.append(ds.GetRasterBand(i+1).ReadAsArray())
    cube = np.dstack(bands)
    p1 = [0.4]
    p2 = [0.2]
    #p1 = [p/100 for p in range(10, 100, 10)]
    #p2 = [p/100 for p in range(10, 100, 10)]
    print(p1, p2)
    for i in p1:
        for j in p2:
            classified = InstSegm(cube[:,:,0], cube[:,:,1], i, j)
            storPath = f'/data/fields/output/predictions/FORCE/segmented/{path.split('/')[-1].split('.')[0]}_segm_{i}_{j}.tif'
            #exportNPtoVRTextent(classified, path, storPath)
            vectorizeRaster(storPath, f'/data/fields/output/predictions/FORCE/segmented_polygons/{path.split('/')[-1].split('.')[0]}_segm_poly_{i}_{j}.shp')

         




/home/potzschf/mambaforge/envs/workhorse/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


[0.4] [0.2]
Raster successfully vectorized and stored at: /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/segmented_polygons/512_80_chipsvrt_segm_poly_0.4_0.2.shp
[0.4] [0.2]
Raster successfully vectorized and stored at: /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/segmented_polygons/512_80_masked_chipsvrt_segm_poly_0.4_0.2.shp
